In [2]:
import math
import random
from collections import defaultdict
import deuces
import itertools

In [204]:
import deuces.lookup


class Card:
    def __init__(self, rank:str, suit:str, value:int):
        self.rank = rank
        self.suit = suit
        self.value = value
    
    def show(self):
        print(f"{self.rank} of {self.suit}: {self.value} chips")
    
    @staticmethod
    def as_str(card):
        return card.rank+card.suit
    
    @staticmethod
    def as_str_list(card_list):
        return [card.rank+card.suit for card in card_list]
    
    @staticmethod
    def convert_deuces_ints_to_strs(card_list):
        return [deuces.Card.int_to_str(card) for card in card_list]

    @staticmethod
    def convert_card_strs_to_deuces(card_list):
        return [deuces.Card.new(card.as_str(card)) for card in card_list]

class Blind_Manager:
    ante_vals = [300, 800, 2000, 5000, 11000, 20000, 35000, 50000]
    ante_endless_vals = [1.1e+5, 5.6e+5, 7.2e+6, 3e+8, 4.7e+10, 2.9e+13, 7.7e+16, 8.6e+20]
    ante_vals += ante_endless_vals
    blind_mult = [1, 1.5, 2]
    blind_values = [int(round[0]*round[1]) for round in list(itertools.product(ante_vals,blind_mult))]

    def __init__(self):
        pass
    pass

class Deck_Manager:
    evaluator = deuces.Evaluator()
    suits = ['d', 'h', 'c', 's']
    rank_values = {
        'A': 11,
        'K': 10,
        'Q': 10,
        'J': 10,
        'T': 10,
        '9': 9,
        '8': 8,
        '7': 7,
        '6': 6,
        '5': 5,
        '4': 4,
        '3': 3,
        '2': 2
    }

    def __init__(self):
        self.deck = []
        for suit in self.suits:
            for rank, value, in self.rank_values.items():
                self.deck.append(Card(rank, suit, value))
        self.shuffle_deck()
        self.first_hand_dealt = False

    def show_deck(self):
        if len(self.deck) == 0: print('No cards in deck!')
        else: 
            for card in self.deck:
                card.show_card()
    
    def shuffle_deck(self):
        random.shuffle(self.deck)
        self.deck_queue = (card for card in self.deck)
        self.first_hand_dealt = False
    
    def draw_card(self):
        try:
            return next(self.deck_queue)
        except StopIteration:
            return None
    
    def draw_cards(self, n:int=8):
        """
        Return list of Card objects until Card generator empty
        """
        card_list = []
        if self.first_hand_dealt == False: 
            cards_to_deal = 8
            self.first_hand_dealt = True
        else: 
            cards_to_deal = n
        for i in range(cards_to_deal):
            new_card = self.draw_card()
            if new_card != None:
                card_list.append(new_card)
            else:
                break
        return card_list
    
    def draw_and_submit(self):
        card_list = self.draw_cards()
        # for card in card_list:
            
        pass
    
    def eight_card_eval(self, cards:list[int]):
        # TODO check for high card, pair, 2 pair, 3 of a kind, 4 of a kind if len(cards)<5
        if len(cards) >= 5:
            minimum = deuces.lookup.LookupTable.MAX_HIGH_CARD # 7462
            minimum_combo = []
            all5cardcombos = itertools.combinations(cards, 5)
            for combo in all5cardcombos:
                score = self.evaluator._five(combo)
                if score < minimum:
                    minimum = score
                    minimum_combo = Card.convert_deuces_ints_to_strs(combo)
            return [minimum, minimum_combo]
        else:
            raise Exception(ValueError)
    
    def get_best_hand(self, cards):
        best_hand_type_int, best_cards = self.eight_card_eval(cards)
        best_hand_type = self.evaluator.class_to_string(self.evaluator.get_rank_class(best_hand_type_int))
        # TODO if hand_type == 5 card hand type, return as is; else remove/keep remaining cards
        if best_hand_type in ['Straight Flush', 'Full House', 'Flush', 'Straight']:
            return [best_hand_type, best_cards]
        else:
            print('not 5 card hand')
            return [best_hand_type, best_cards]


In [253]:
deck_m = Deck_Manager()
eval_m = deuces.Evaluator()

In [267]:
test_cards = deck_m.draw_cards(5)
test_cards


In [268]:
print(Card.as_str_list(test_cards))

['7c', '5s', '2d', '4h']


In [266]:
test_cards_str = Card.convert_card_strs_to_deuces(test_cards)
test_str, test_card_list2 = deck_m.get_best_hand(test_cards_str)
print(test_str, test_card_list2)

Flush ['2h', 'Qh', '7h', '6h', 'Jh']


In [36]:
test1 = [300, 800, 2000, 5000, 11000, 20000, 35000, 50000, 1.1e+5, 5.6e+5, 7.2e+6, 3e+8, 4.7e+10, 2.9e+13, 7.7e+16, 8.6e+20]
test1 += [1.1e+5, 5.6e+5, 7.2e+6, 3e+8, 4.7e+10, 2.9e+13, 7.7e+16, 8.6e+20]
test3 = [1, 1.5, 2]

test_list = list(itertools.product(test1,test3))
[int(i[0]*i[1]) for i in test_list]

[300,
 450,
 600,
 800,
 1200,
 1600,
 2000,
 3000,
 4000,
 5000,
 7500,
 10000,
 11000,
 16500,
 22000,
 20000,
 30000,
 40000,
 35000,
 52500,
 70000,
 50000,
 75000,
 100000,
 110000,
 165000,
 220000,
 560000,
 840000,
 1120000,
 7200000,
 10800000,
 14400000,
 300000000,
 450000000,
 600000000,
 47000000000,
 70500000000,
 94000000000,
 29000000000000,
 43500000000000,
 58000000000000,
 77000000000000000,
 115500000000000000,
 154000000000000000,
 860000000000000000000,
 1290000000000000000000,
 1720000000000000000000,
 110000,
 165000,
 220000,
 560000,
 840000,
 1120000,
 7200000,
 10800000,
 14400000,
 300000000,
 450000000,
 600000000,
 47000000000,
 70500000000,
 94000000000,
 29000000000000,
 43500000000000,
 58000000000000,
 77000000000000000,
 115500000000000000,
 154000000000000000,
 860000000000000000000,
 1290000000000000000000,
 1720000000000000000000]